<a href="https://colab.research.google.com/github/CamilleGreen5/Deep_Learning_with_Python/blob/master/Noise_speech_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
!pip install tdqm
!pip install pydub

  Created wheel for tdqm: filename=tdqm-0.0.1-cp36-none-any.whl size=1322 sha256=0c69c69bf175064599d2638469dc1b99dd3be3d121d4330396d6b8761ed0e5de
  Stored in directory: /root/.cache/pip/wheels/d5/c0/4b/a98405d8b13071ab9a2be7beeef6820d26349ee46651c16a0d
Successfully built tdqm


In [10]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import requests
from scipy.io.wavfile import read, write
from scipy.signal import stft, istft
import glob
from pydub import AudioSegment

In [4]:
!wget https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-5.1-2020-06-22/fr.tar.gz

--2020-09-14 07:08:58--  https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-5.1-2020-06-22/fr.tar.gz
Resolving voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)... 52.218.216.243
Connecting to voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)|52.218.216.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16959458032 (16G) [application/octet-stream]
Saving to: ‘fr.tar.gz’

fr.tar.gz           100%[===================>]  15.79G  36.3MB/s    in 7m 43s  

2020-09-14 07:16:41 (35.0 MB/s) - ‘fr.tar.gz’ saved [16959458032/16959458032]



In [5]:
!tar -xf fr.tar.gz

In [ ]:
!ls cv-corpus-5.1-2020-06-22/fr/clips

In [63]:
list_mp3_files = glob.glob("./cv-corpus-5.1-2020-06-22/fr/clips/*.mp3")
print(len(list_mp3_files))

450077


In [65]:
train_mp3_data = list_mp3_files[:500]
test_mp3_data = list_mp3_files[500:600]

In [71]:
from tqdm import tqdm

labels = []
samples = []

for i in tdqm(len(train_mp3_data)):

    path_to_mp3_file = train_mp3_data[i]
    # load mp3 sound to array
    loaded_sound = AudioSegment.from_mp3(path_to_mp3_file)
    array_sound = np.array(loaded_sound.get_array_of_samples(), np.float64)

    # normalize array
    mean = np.mean(array_sound, axis=0)
    std = np.std(array_sound)
    array_sound -= mean
    array_sound /= std

    # convert label to to F_T_space
    f, t, array_stft = stft(array_sound, fs=48000, nperseg=1000)
    label_abs_stft = np.abs(array_stft)

    labels.append(label_abs_stft)


    # add random noise
    max_1 = max(array_sound)
    noise = np.random.normal(0, 0.75, len(array_sound))
    array_sound_noisy = np.add(noise, array_sound)
    max_2 = max(array_sound_noisy)
    array_sound_noisy *= max_1/max_2

    # convert sample to to F_T_space
    f, t, array_stft = stft(array_sound_noisy, fs=48000, nperseg=1000)
    sample_abs_stft = np.abs(array_stft)

    samples.append(sample_abs_stft)


NameError: ignored

In [21]:
samplerate = 44100; fs = 10
t = np.linspace(0., 1., samplerate)
amplitude = np.iinfo(np.int16).max
data = amplitude * np.sin(2. * np.pi * fs * t)
write("example.wav", samplerate, data)

In [32]:
# load mp3 sound to array
mp3_sound = AudioSegment.from_mp3("./original/common_voice_fr_18157595.mp3")
array_sound_mp3 = np.array(mp3_sound.get_array_of_samples())

# save original to wav
path_to_wav_file = './noised/noised_sound0.wav'
write(path_to_wav_file, 48000, array_sound_mp3)

# save to mp3 from wav
mp3_sound_save = AudioSegment.from_wav(path_to_wav_file)
mp3_sound_save.export('./original/noised_sound0.mp3', format='mp3')

# load wav sound to array
samplerate, wav_sound = read("./noised/noised_sound0.wav")
array_sound_wav = np.array(wav_sound, np.float64)

# # plot
# plt.plot([i for i  in range(len(array_sound_wav))], array_sound_wav, 'b')
# plt.show()
# plt.plot([i for i  in range(len(array_sound_mp3))], array_sound_mp3, 'r')
# plt.show()


In [ ]:
for i, path_to_mp3_file in enumerate(list_mp3_files[1:2]):

    # load mp3 sound to array
    loaded_sound = AudioSegment.from_mp3(path_to_mp3_file)
    array_sound = np.array(loaded_sound.get_array_of_samples(), np.float64)

    # normalize array
    mean = np.mean(array_sound, axis=0)
    std = np.std(array_sound)
    array_sound -= mean
    array_sound /= std

    # add random noise
    max_1 = max(array_sound)
    noise = np.random.normal(0, 0.75, len(array_sound))
    array_sound_noisy = np.add(noise, array_sound)
    max_2 = max(array_sound_noisy)
    array_sound_noisy *= max_1/max_2
    
    # save original to wav
    array_sound_noisy_to_save = array_sound_noisy * std
    array_sound_noisy_to_save += mean
    array_sound_noisy_to_save = np.array(array_sound_noisy_to_save, np.int16)
    path_to_wav_file = './noised/noised_sound0.wav'
    write(path_to_wav_file, 48000, array_sound_noisy_to_save)

    # plot original and noisy
    plt.plot([i for i  in range(len(array_sound))], array_sound_noisy, 'r')
    plt.plot([i for i  in range(len(array_sound))], array_sound, 'b')
    plt.show()

    # convert sound to to F_T_space
    f, t, array_stft = stft(array_sound_noisy, fs=48000, nperseg=1000)
    array_abs_stft = np.abs(array_stft)

    plt.pcolormesh(t, f, array_abs_stft)
    plt.title('STFT Magnitude')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

    # convert back to sound from F_T_space
    t, desnoised_array_sound = istft(array_stft, fs=48000, nperseg=1000)
    desnoised_array_sound = np.array(desnoised_array_sound)
    desnoised_array_sound *= std
    desnoised_array_sound += mean
    desnoised_array_sound = np.array(desnoised_array_sound, np.int16)

    # save reconverted sound to wav
    path_to_wav_file = './denoised/denoised_sound' + str(i) + '.wav'
    write(path_to_wav_file, 48000, desnoised_array_sound)
    

In [69]:
import time
import sys

toolbar_width = 40

# setup toolbar
sys.stdout.write("[%s]" % (" " * toolbar_width))
sys.stdout.flush()
sys.stdout.write("\b" * (toolbar_width+1)) # return to start of line, after '['

for i in range(toolbar_width):
    time.sleep(0.1) # do real work here
    # update the bar
    sys.stdout.write("-")
    sys.stdout.flush()
sys.stdout.write("]\n") # this ends the progress bar

[                                        ]----------------------------------------]
